# Load Model

In [ ]:
model_path = 'model'
model_ext = 'keras'

In [ ]:
model = tf.keras.models.load_model(model_path+model_ext)

# Convert to Onnx

In [ ]:
spec = (tf.TensorSpec((None, 123, 123, 3), tf.float32),)
model_proto, external_tensor_storage = tf2onnx.convert.from_keras(model, input_signature=spec)

In [ ]:
with open(model_path+".onnx", "wb") as f:
    f.write(model_proto.SerializeToString())

# Convert to TFLite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [ ]:
with open(model_path+".tflite", 'wb') as f:
    f.write(tflite_model)